In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import time
from sklearn.datasets import fetch_kddcup99
from sklearn.preprocessing import StandardScaler

from pyspark import SparkContext, SparkConf
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col

from utils import compute_centroidDistances, get_clusterId, get_minDistance

## Import algorithms functions

In [2]:
def kMeansPlusPlus_init(
    data: npt.NDArray,
    k: int,
    weights: npt.NDArray = np.array([])
) -> npt.NDArray:
    
    #Standard kMeans++ initialization method:
    #given `data` (eventually weighted), returns `k` cluster centroids
    
    if weights.shape[0] == 0:
        weights = np.ones(shape=(data.shape[0],))
    
    centroids = data[np.random.randint(0, data.shape[0]),:].reshape(1, -1) # reshaping for easier stacking
    
    while (centroids.shape[0] < k):
        # since the original functions are made for map
        # we need to loop over the data
        minDistance_array = np.array(
            [get_minDistance(compute_centroidDistances(datum, centroids)) for datum in data]
        ) * weights # multiplyling by the weight simulates multiple copies of the same datum
        total_minDistance = np.sum(minDistance_array)
        # sampling probability proportional to minDistance
        new_centroid_idx = np.random.choice(minDistance_array.shape[0], size = 1, p = minDistance_array / total_minDistance)
        new_centroid = data[new_centroid_idx,:].reshape(1, -1)

        # edge case in which the same centroid is selected twice:
        # redo the iteration without saving the centroid
        if any(np.array_equal(new_centroid, row) for row in centroids): continue
        centroids = np.concatenate((centroids, new_centroid), axis = 0)

    return centroids

In [3]:
def kMeansNaive(
    data: npt.NDArray,
    centroids: npt.NDArray,
    epochs: int = 5
) -> npt.NDArray:
    """
    Standard kMeans algorithm:
    given `data`, updates the (k) `centroids` for `epochs` times,
    improving the clustering each time
    """
    k = centroids.shape[0]
    for _ in range(epochs):
        assignments = np.array(
            [get_clusterId(compute_centroidDistances(x, centroids)) for x in data]
        )
        centroids = np.array(
            [np.mean(data[assignments==i,:], axis = 0) for i in range(k)]
        )
    return centroids

In [4]:
def kMeansParallel_init(
    data_rdd: RDD,
    k: int,
    l: float
) -> npt.NDArray:
    """
    kMeans|| initialization method:
    returns `k` good `centroids`.
    `l` controls the probability of each point
    in `data_rdd` of being sampled as a pre-processed centroid.
    """

    centroids = np.array(
        data_rdd.takeSample(num=1, withReplacement=False)
    )
    # centroid = data_rdd.takeSample(num=1, withReplacement=False, seed=42)[0]
    # centroids = np.array([centroid])

    
    minDistance_rdd = data_rdd \
        .map(lambda x: (x, get_minDistance(compute_centroidDistances(x, centroids)))) \
        .persist()

    cost = minDistance_rdd \
        .map(lambda x: x[1]) \
        .sum()

    iterations = int(np.ceil(np.log(cost))) if (cost > 1) else 1
    for _ in range(iterations):
        new_centroids = np.array(
            minDistance_rdd \
                .filter(lambda x: np.random.rand() < np.min((l * x[1] / cost, 1))) \
                .map(lambda x: x[0]) \
                .collect()
        )
        # edge case in which no new centroid is sampled:
        # this avoids the following `np.concatenate` to fail
        if len(new_centroids.shape) < 2:
            continue

        minDistance_rdd.unpersist()
        centroids = np.unique(
            np.concatenate((centroids, new_centroids), axis = 0), 
            axis = 0
        )

        minDistance_rdd = data_rdd \
            .map(lambda x: (x, get_minDistance(compute_centroidDistances(x, centroids)))) \
            .persist()
        cost = minDistance_rdd \
            .map(lambda x: x[1]) \
            .sum()
    
    minDistance_rdd.unpersist()
    clusterCounts = data_rdd \
        .map(lambda x: (get_clusterId(compute_centroidDistances(x, centroids)), 1)) \
        .countByKey()
    
    clusterCounts = np.array([w[1] for w in clusterCounts.items()])
    centroids = kMeansNaive(
        centroids, 
        kMeansPlusPlus_init(centroids, k, clusterCounts)
    )
    
    return centroids

In [5]:
def miniBatchKMeans(
    data_rdd: RDD,
    centroids: npt.NDArray,
    iterations: int = 10,
    batch_fraction: float = 0.1
) -> npt.NDArray:
    k = centroids.shape[0]
    clusterCounters = np.zeros((k,)) # 1 / learning_rate
    for iter in range(iterations):
        miniBatch_rdd = data_rdd \
            .sample(withReplacement=False, fraction=batch_fraction)
        miniBatch_rdd = miniBatch_rdd \
            .map(lambda x: (get_clusterId(compute_centroidDistances(x, centroids)), 1, x)) \
            .persist()
        
        # counting how many assigments per cluster
        clusterCounts_dict = miniBatch_rdd \
            .map(lambda x: (x[0], x[1])) \
            .countByKey()
        clusterCounts = np.array(
            [clusterCounts_dict[i] if i in clusterCounts_dict.keys() else 0 for i in range(k)]
        )
        clusterCounters += clusterCounts
        
        # edge case in which a cluster has no assignments:
        # if also its counter is zero the whole iteration is repeated
        if any(np.isclose(v, 0) for v in clusterCounters): 
            iter -= 1
            miniBatch_rdd.unpersist()
            continue
        # otherwise its count will be set to 1 to avoid division by 0 in the update step
        clusterCounts = np.where(clusterCounts >= 1, clusterCounts, 1)

        # summing all points assigned to the same cluster
        # (in the update step this will be divided by the counts 
        # in order to get the mean for every cluster).
        # A dict is used for convenience and consistency with clusterCounts
        """clusterSums_dict = dict(miniBatch_rdd \
            .map(lambda x: (x[0], x[2])) \
            .reduceByKey(lambda x, y: x + y) \
            .collect()
        )
        # edge case in which a cluster has no assignments:
        # the centroid is returned instead of 0 
        # (which would have been the sum of its assigned points) 
        # in order to not update its position 
        # (note how the terms cancel out in the update step)
        clusterSums = np.array([
            np.array(clusterSums_dict[i]) if i in clusterSums_dict else centroids[i, :]
            for i in range(k)
        ])"""
        # summing all points assigned to the same cluster
        clusterSums_dict = dict(
            miniBatch_rdd
                .map(lambda x: (x[0], np.array(x[2])))  # FORZA conversione a numpy
                .reduceByKey(lambda x, y: x + y)
                .collect()
        )
        
        clusterSums = np.stack([
            clusterSums_dict[i] if i in clusterSums_dict else centroids[i, :]
            for i in range(k)
        ])



        # update step: c <- (1 - eta) * c + eta * x_mean
        # (note x_mean = x_sums / c_count)
        centroids = (1 - 1 / clusterCounters).reshape(-1, 1) * centroids + \
                    (1 / (clusterCounters * clusterCounts)).reshape(-1, 1) * clusterSums
        
        miniBatch_rdd.unpersist()
        
    return centroids

## Start a Spark session

In [6]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("kMeansParallel") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

sc = spark.sparkContext
sc.addPyFile("utils.py") 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/09 11:01:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load and preprocess the dataset

In [7]:
"""# Generated dataset

# Parameters for toy dataset
num_points_per_cluster = 50
num_clusters = 10
dim = 3  # 2D points for easy visualization
spread = 0.5
seed = 42
np.random.seed(seed)

centers = np.random.uniform(-10, 10, (num_clusters, dim))
data = np.concatenate(
    [center + spread * np.random.randn(num_points_per_cluster, dim) for center in centers],
    axis = 0
)
data_rdd = sc.parallelize([row for row in data])

data_rdd = data_rdd.persist()"""

'# Generated dataset\n\n# Parameters for toy dataset\nnum_points_per_cluster = 50\nnum_clusters = 10\ndim = 3  # 2D points for easy visualization\nspread = 0.5\nseed = 42\nnp.random.seed(seed)\n\ncenters = np.random.uniform(-10, 10, (num_clusters, dim))\ndata = np.concatenate(\n    [center + spread * np.random.randn(num_points_per_cluster, dim) for center in centers],\n    axis = 0\n)\ndata_rdd = sc.parallelize([row for row in data])\n\ndata_rdd = data_rdd.persist()'

The first dataset we would like to test is a synthetic GaussMixture. To generate it, we sampled kcenters from a 15-dimensional spherical Gaussian distribution with mean at the origin and variance R∈{1,10,100}. We then added points from Gaussian distributions of unit variance around each center. Given the k centers, this is a mixture of k spherical Gaussians with equal weights.

ref{paper kmeans||}

In [8]:
# Generated dataset

# Parameters
def gauss_mixture(
    num_points_per_cluster = 50,  
    num_clusters = 10, # k centers
    dim = 15,                  
    R = 10 # center variances
):

    seed = 42
    np.random.seed(seed)
    
    # Centers generation N(0, R*I)
    centers = np.random.normal(loc=0, scale=np.sqrt(R), size=(num_clusters, dim))
    
    # Point generation N(center, I) for each cluster
    data = np.concatenate(
        [center + np.random.randn(num_points_per_cluster, dim) for center in centers],
        axis=0
    )
    
    data_rdd = sc.parallelize([row for row in data])
    data_rdd = data_rdd.persist()
    
    return data_rdd

In [9]:
# try to generate data
data_rdd = gauss_mixture()

In [10]:
num_elements = data_rdd.count()
element_dim = len(data_rdd.first())
print(f"Shape of RDD object: ({num_elements}, {element_dim})")

Shape of RDD object: (500, 15)


In [11]:
# Convert to numpy array
data_list = data_rdd.collect()
data_array = np.array(data_list)

In [12]:
# Real dataset

# Change percent10 to 'False' to fetch the full dataset (4M rows)
# for local works is better to leave it as 'True'
kdd = fetch_kddcup99(shuffle=True, percent10=True) 
kdd_data = kdd.data

# Remove string features and standardize them
data_kdd = np.delete(kdd_data,np.arange(1,4,1),axis = 1) 
scaler_kdd = StandardScaler()
data_kdd = scaler_kdd.fit_transform(data_kdd)

# parallelize
data_rdd_kdd = sc.parallelize([row for row in data_kdd])
# data_rdd = sc.parallelize(np.array(data).tolist(), numSlices=16)
data_rdd_kdd = data_rdd_kdd.persist()

In [13]:
num_elements = data_rdd_kdd.count()
element_dim = len(data_rdd_kdd.first())
print(f"Shape of RDD object: ({num_elements}, {element_dim})")

25/09/09 11:02:06 WARN TaskSetManager: Stage 3 contains a task of very large size (20239 KiB). The maximum recommended task size is 1000 KiB.
25/09/09 11:02:21 WARN TaskSetManager: Stage 4 contains a task of very large size (20239 KiB). The maximum recommended task size is 1000 KiB.
[Stage 4:>                                                          (0 + 1) / 1]

Shape of RDD object: (494021, 38)


In [14]:
k = 6
l = 0.5*k
centroids = kMeansParallel_init(data_rdd, k, l)

print(centroids.shape)

(6, 15)


In [15]:
final_centroids = miniBatchKMeans(data_rdd, centroids)

print(final_centroids.shape)

(6, 15)


## Benchmark testing

In [16]:
def cost_function(data, centroids):
    """
    Compute cost function: square distance between data and centroids
    data: (N, d)
    centroids: (k, d)
    """

    distances = np.linalg.norm(data[:, None, :] - centroids[None, :, :], axis=2)**2
    min_distances = np.min(distances, axis=1)
    
    # sum the distances to get the final cost
    cost = np.sum(min_distances)

    # return float(cost)
    return cost

In [17]:
def analysis(data_array, data_rdd, r, k, iterations=10, batch_fraction=0.1):
    results = []

    # 1 - Random initialization
    start = time.time()
    centroidsRandom = data_array[np.random.choice(data_array.shape[0], size=k, replace=False)]
    init_time = time.time() - start
    seed_cost = cost_function(data_array, centroidsRandom)
    
    final_centroids = miniBatchKMeans(data_rdd, centroidsRandom, iterations, batch_fraction)
    final_cost = cost_function(data_array, final_centroids)
    
    results.append({
        "method": "random",
        "k": k, 
        "l": np.nan,
        "r": r, 
        "initialization_time": init_time,
        "seed": seed_cost,
        "final": final_cost
    })

    # 2 - k-means++
    start = time.time()
    centroidsPlusPlus = kMeansPlusPlus_init(data_array, k)
    init_time = time.time() - start
    seed_cost = cost_function(data_array, centroidsPlusPlus)
    final_centroids = miniBatchKMeans(data_rdd, centroidsPlusPlus, iterations, batch_fraction)
    final_cost = cost_function(data_array, final_centroids)
    results.append({
        "method": "kmeans++",
        "k": k,
        "l": np.nan,
        "r": r, 
        "initialization_time": init_time,
        "seed": seed_cost,
        "final": final_cost
    })

    # 3 - Parallel l=0.5k
    start = time.time()
    centroidsParallel1 = kMeansParallel_init(data_rdd, k=k, l=0.5*k)
    init_time = time.time() - start
    seed_cost = cost_function(data_array, centroidsParallel1)
    final_centroids = miniBatchKMeans(data_rdd, centroidsParallel1, iterations, batch_fraction)
    final_cost = cost_function(data_array, final_centroids)
    results.append({
        "method": "kmeans|| (l=k/2)",
        "k": k,
        "l": 0.5*k,
        "r": r, 
        "initialization_time": init_time,
        "seed": seed_cost,
        "final": final_cost
    })

    # 4 - Parallel l=2k
    start = time.time()
    centroidsParallel2 = kMeansParallel_init(data_rdd, k=k, l=2*k)
    init_time = time.time() - start
    seed_cost = cost_function(data_array, centroidsParallel2)
    final_centroids = miniBatchKMeans(data_rdd, centroidsParallel2, iterations, batch_fraction)
    final_cost = cost_function(data_array, final_centroids)
    results.append({
        "method": "kmeans|| (l=2k)",
        "k": k,
        "l": 2*k,
        "r": r, 
        "initialization_time": init_time,
        "seed": seed_cost,
        "final": final_cost
    })

    df = pd.DataFrame(results)
    return df

Paper analysis on Gaussian mixture

In [ ]:
R = [1, 10, 100]

# generate data with Gaussian mixture changing R
for RR in R:
    data_rdd = gauss_mixture(R=RR)
    # Convert to numpy array
    data_list = data_rdd.collect()
    data_array = np.array(data_list)

    df = analysis(data_array, data_rdd, r=5, k=50)
    print("R =", RR, "\n")
    print(df)